## Jupyter Lab Workflow
Jupyter is a computing environment supporting several interpreted programming languages, cell-by-cell execution, and inline output, including rich tables and charts. It is most often  paired with python. It is ideal for quick, iterative data exploration and visualization. All tools and libraries used in this short demonstration are free and open-source. 
* www.python.org : Underlying programming language
* www.jupyter.org : Environment used for interactive execution
* www.pandas.pydata.org : Python library used to analyze tabular data
* www.plotly.com : Python library used to generate rich, interactive charts

If you are new to Jupyter, you probably want to start by running all cells:

<img src="img/RunAll.png" width=800/>

In case you later change the contents of a single cell, you can run the code inside it again with the following button:

<img src="img/RunCell.png" width=800/>


AUTHORS: 
The first version of this code was created by Cole Hudson from Shell Eco-marathons Data & Technology Team. Lukas Ammann from the same team adapted it to the latest data format that is provided to students after each attempt.

DISCLAIMER: 
The python code provided by the authors is given "as is" without any warranties. Users of the code assume full responsibility for their actions and agree to hold the authors harmless from any liability. The authors do not guarantee the accuracy or completeness of the Materials and are not liable for any damages or losses incurred from their use. Users are responsible for verifying information independently. The authors are not responsible for any third-party materials and reserves the right to modify or terminate the availability of the Materials.
All data included in this program has not been formally verified, and therefore the authors are not responsible for the data accuracy. For informational purposes only. Telemetry data is not used to calculate, nor does it replace official results. NO protests and NO modifications to official results will be allowed based on telemetry data. The code is provided "as is" with no responsibility. 

In [ ]:
import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

pd.options.plotting.backend = 'plotly'

# pio.templates.default = 'plotly_dark'

### Loading Data
Pandas library is used to read the tabular data using the `read_csv` method. You can read in your own data by uploading your own datafile to the notebook and changing the filename in the `read_csv` method to the name of your datafile. Remember to rerun all cells if you import your own data in order to have plots based on your data.

<img src="img/ImportFile.png" width=400/>


In [1]:
df = pd.read_csv('../Data/Sample/ICE_sample_data.csv', sep=',', low_memory=False)

NameError: name 'pd' is not defined

Little bit of cleanup is then done. 
* Columns are all lowercased, to make retrieval by name a bit easier
* We reset the index, just-in-case something is out of order.
* We then "forward fill" the dataframe, to fill the blank (null) values in the data with the most recent numeric value in each column

In [ ]:
df.columns = [col.lower() for col in df.columns]
df.reset_index(drop=True, inplace=True)
df.ffill(inplace=True)
df.head(10)

,rec_id,obc_timestamp,lap_obc_timestamp,dist,lap_dist,gps_utctime,gps_latitude,gps_longitude,gps_speed,lfm_temperature,...,lap_gfs_netgasvolume,jm3_netjoule,lap_jm3_netjoule,jm3_voltage,jm3_current,volt_voltage,volt_difference,lap_lap,start_lap,finish_lap
0,0,1697005248,0.0,0.000000,0.000000,06.20.46,13.154726,77.699563,2.5,30.2,...,NaN,627.757,0.000,12490,2034,NaN,NaN,1,1,0
1,1,1697005248,0.1,2.568304,2.568304,06.20.47,13.154728,77.699540,7.7,30.2,...,NaN,632.481,4.724,12497,1898,NaN,NaN,1,1,0
2,2,1697005248,0.2,2.568304,2.568304,06.20.47,13.154728,77.699540,7.7,30.2,...,NaN,632.481,4.724,12497,1898,NaN,NaN,1,1,0
3,3,1697005248,0.3,2.568304,2.568304,06.20.47,13.154728,77.699540,7.7,30.2,...,NaN,637.338,9.581,12496,1945,NaN,NaN,1,1,0
4,4,1697005248,0.4,2.568304,2.568304,06.20.47,13.154728,77.699540,7.7,30.2,...,NaN,637.338,9.581,12496,1945,NaN,NaN,1,1,0
5,5,1697005248,0.5,2.568304,2.568304,06.20.47,13.154728,77.699540,7.7,30.2,...,NaN,642.287,14.530,12493,1969,NaN,NaN,1,1,0
6,6,1697005248,0.6,2.568304,2.568304,06.20.47,13.154728,77.699540,7.7,30.2,...,NaN,642.287,14.530,12493,1969,NaN,NaN,1,1,0
7,7,1697005248,0.7,2.568304,2.568304,06.20.47,13.154728,77.699540,7.7,30.2,...,NaN,647.046,19.289,12495,1913,NaN,NaN,1,1,0
8,8,1697005249,0.8,2.568304,2.568304,06.20.47,13.154728,77.699540,7.7,30.2,...,NaN,647.046,19.289,12495,1913,NaN,NaN,1,1,0
9,9,1697005249,0.9,2.568304,2.568304,06.20.47,13.154728,77.699540,7.7,30.2,...,NaN,651.702,23.945,12497,1862,NaN,NaN,1,1,0


### Data Visualization
We'll quickly plot the available GPS trace on top of the start, finish, and lap lines. Looks like all the data is here!

In [ ]:
df.plot(x='gps_longitude', y='gps_latitude')

We can also plot the data lap-by-lap by slightly adjusting the code:

In [ ]:
df.plot(x='gps_longitude', y='gps_latitude', color='lap_lap')

We can now do some analysis in the context of laps, aggregate data and compare values based data for each lap.

Take note of the [Telemetry Wiki](https://telemetry.sem-app.com/wiki/doku.php/telemetry_data/channel_descriptions). If you are wondering what kind of data is stored under a label such as `'lfm_integratedCorrFlow'` and what unit it has, you will find that information there.

In [ ]:
df.plot(x='lap_dist', y='lfm_instantflow', color='lap_lap')

In [ ]:
df.plot(x='lap_dist', y='lap_lfm_integratedcorrflow', color='lap_lap')

In [ ]:
df.plot(x='gps_longitude', y='gps_latitude', color='lfm_instantflow', kind='scatter', facet_col='lap_lap', facet_col_wrap=4)

In [ ]:
df.plot(x='lap_dist', y='lap_jm3_netjoule', color='lap_lap')

Maybe we are interested in comparing two laps, like the 4th and 5th lap that are compared here:

In [ ]:
df.loc[df.lap_lap.isin([1, 5])].plot(x='lap_dist', y='lap_lfm_integratedcorrflow', color='lap_lap')

In [ ]:
df.loc[df.lap_lap.isin([1, 5])].plot(x='lap_dist', y='lfm_instantflow', color='lap_lap')

In [ ]:
df.loc[df.lap_lap.isin([1, 5])].plot(x='gps_longitude', y='gps_latitude', color='lfm_instantflow', kind='scatter', facet_col='lap_lap')

In [ ]:
df.loc[df.lap_lap.isin([1, 5])].plot(x='lap_dist', y='lap_jm3_netjoule', color='lap_lap')

### Calculating new data
Maybe we want plot a physical quantity which isn't in the data set by default. We can add a new collum to the data using the syntax `df['new_label'] = df['label1']/df['label2']` which goes row by row through the data, on each row setting the value of `'new_label'` according to the values of `'label1'` and `'label2'` on that row. We can use that to calculate the instantanoues Power flowing through the Joulmeter.

As the data is in mV and mA, as can be seen on the [Telemetry Wiki](https://telemetry.sem-app.com/wiki/doku.php/telemetry_data/channel_descriptions), we will have to divide the result by 1 million if we want the result to be in Watts.

In [ ]:
df['jm3_power'] = df['jm3_voltage']*df['jm3_current']/10**6

df.plot(x='lap_dist', y='jm3_power', color='lap_lap')

If we want to calculate new data, that is based on the values of previous cells, we can't use the simple syntax from before. Instead we have to loop through the data as the following code shows by the example of calculating the power using the difference of accumulated joules over time. Note that calculations done this way take a lot longer than before.

In the code the variable `window_len` defines from how many rows back data is needed to calculate the value of the current row. This is needed to ensure that only data that actually exists is accessed for the calculations. As jm3_netjoule is only updated every 200 ms, window_len must be at least 2, but it can be increased to calculate the mean over a longer time.

In [ ]:
#initalising new collum:
df['jm3_power_alternative'] = .0

window_len = 2

#manually adding data to our new collum
for ind in range(window_len, len(df)):
    ind_prev = ind-window_len
    df.loc[ind, 'jm3_power_alternative'] = (df.loc[ind, 'jm3_netjoule'] - df.loc[ind_prev, 'jm3_netjoule']) / (df.loc[ind, 'lap_obc_timestamp'] - df.loc[ind_prev, 'lap_obc_timestamp'])
df.plot(x='lap_dist', y='jm3_power_alternative', color='lap_lap')

Let's do some more complicated data analysis now. First we calculate two new data collums. The values are calculated as moving average.

|**Label**            |**Unit** |**Description**                                      |
|----------------------|----------|------------------------------------------------------|
|**acceleration**     |m/s²     |How much the speed changes per second                |
|**consumption**      |ml/s     |How much fuel is used per second                     |

In [ ]:
df['acceleration'] = 0.
df['consumption'] = 0.

window_len = 30

for ind in range(window_len, len(df)):
    ind_prev = ind-window_len
    df.loc[ind, 'acceleration'] = (df.loc[ind, 'gps_speed'] - df.loc[ind_prev, 'gps_speed'])/(df.loc[ind, 'obc_timestamp'] - df.loc[ind_prev, 'obc_timestamp'])
    df.loc[ind, 'consumption'] = (df.loc[ind, 'lfm_integratedcorrflow'] - df.loc[ind_prev, 'lfm_integratedcorrflow'])/(df.loc[ind, 'obc_timestamp'] - df.loc[ind_prev, 'obc_timestamp'])

We can plot what we just calculated. Here is a plot of the values we calculated before.

Especially consider the following points that we can see from the example data:
* If the car is not accelerated (and the motor shut off), the car will slow down approximately 0.8 m/s² when driving at a speed of 30 km/h, and a little less when driving slower. You may use that to calculate how far before a stop you have to stop accelerating.

<img src="img/SlowDown.png" width=600/>

* Also visible are curves that correspond to a single maneuver of accleration (like accelerating from 15 to 35 km/h). This follows from the fact that acceleration and consumption were calculated as moving average.
  
<img src="img/AccelerationManeuvers.png" width=600/>

In [ ]:
df.plot.scatter(x='acceleration', y='gps_speed', color='consumption')

### Final Words
There is more data that we can use to create additional plots. The following command can print the aviable lables. Have fun calculating new data and trying out new plots 
Hint: First you may want to try out a plot with `'lap_dist'` on the x-axis and `'gps_speed'` on the y-axis.

In [ ]:
for col in df.columns:
    print(col)

rec_id
obc_timestamp
lap_obc_timestamp
dist
lap_dist
gps_utctime
gps_latitude
gps_longitude
gps_speed
lfm_temperature
lfm_integratedcorrflow
lfm_instantflow
lap_lfm_integratedcorrflow
gfs_gasflow
gfs_gastemp
gfs_netgasvolume
lap_gfs_netgasvolume
jm3_netjoule
lap_jm3_netjoule
jm3_voltage
jm3_current
volt_voltage
volt_difference
lap_lap
start_lap
finish_lap
jm3_power
jm3_power_alternative
acceleration
consumption
